#Главня функция - predict_class, в ее теле вызываются все остальные функции.
#Изображение загружается как image объект из библиотеки Pillow
#На вход в обе сети подаются картинки переведенные в numpy массив  
#Размерность входного слоя сети классификации 224/224, сети обрезки 480/480, так что делать фото в качестве full hd и выше смысла нет 


In [ ]:
/# Скачиваем модели
!gdown -q https://drive.google.com/u/0/uc?id=1QqiDCHtJhD2IyxDUpiTHQ2WeYU6WPU62&export=download
!gdown -q https://drive.google.com/u/0/uc?id=182SNXAM6TvNZweuZV8ahtoZ8NJsnE8ru&export=download

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import os
import numpy as np
from PIL import Image

In [ ]:
CROP_MODEL=load_model('Сrop_model.h5') #Путь к кроп модели
CLASS_MODEL=load_model('Class_model.h5') #Путь к классификац модели

In [ ]:
classes = {0: 'Осина внутренний',
           1: 'Внутренний',
           2: 'Береза поперечный',
           3: 'Сорт 4',
           4: 'Сорт 1',
           5: 'Сорт 2',
           6: 'Починка'            
           }

In [ ]:
def load_image(path,target_size=(480,480)):
  image = Image.open(path)
  image = image.resize(target_size)
  return image

In [ ]:
def crop_function(img,initial_size,final_size):
  """
  Функция обрезки фотографии.
  Т.к большу́ю часть фото занимает пол, это может влиять на точность классификации
  Функция берет на вход сырое фото и обрезает лишние пиксели по краям.

  """
  size_scale=initial_size[0]
  img_np = np.array(img)/255
  pred = CROP_MODEL.predict(tf.expand_dims(img_np, axis=0),verbose=0)[0]
  box = pred*initial_size[0]
  box = box.astype('int32') 
  xmin, ymin, xmax, ymax = box[0], box[1], box[2], box[3]
  cr_img = img.crop((xmin,ymin,xmax,ymax))
  cr_img = cr_img.resize(final_size)
  return cr_img  

In [ ]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
def predict_class(img_path):
  """
  Функция предсказания. 
  На вход берет путь к фотографии, обрезает ее и делает предсказание
  На выходе выдает предсказанный класс и вероятность 
  """

  img = load_image(img_path,target_size=(480,480)) #Загружаем фото
  cr_image = np.array(crop_function(img,initial_size=(480,480),final_size=(224,224))) # Обрезаем края
  class_pred = CLASS_MODEL.predict(tf.expand_dims(cr_image, axis=0),verbose=0)[0] #Подаем обрезанное 
  answer=np.argmax(softmax(class_pred))                                           #фото в модель классификайии
  probability=round(softmax(class_pred)[answer]*100,2)
  answer = classes[answer]
  return answer,probability

#Если вывод картинки не нужен, следующую ячейку можно удалить

In [ ]:
import matplotlib.pyplot as plt
img='/content/123-1168_jpg.rf.2882f74b2ee30603cf7213d52c65b79a.jpg'

def print_result(img_path):
  answer,prob = predict_class(img_path)
  print(f'{answer} // {prob}%')
  img = np.array(load_image(img_path))
  plt.imshow(img)
  plt.show()

print_result(img)